<a href="https://colab.research.google.com/github/rubyvanrooyen/astrokat/blob/colab_helper_interface/notebooks/astrokat_catalogue2obsfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AstroKAT catalogue to observation file
The minimum required information for any observation is a list of observation targets specified as one target per line, using comma separated formatting to provide the relevant target information.

For more detail refer to the
[Catalogues to observation files](https://github.com/ska-sa/astrokat/wiki/Catalogues-to-observation-files)
documentation on the
[AstroKAT](https://github.com/ska-sa/astrokat/wiki)
wiki page.


## Installation and setup

In [1]:
!pip install git+https://github.com/ska-sa/astrokat.git

  Cloning https://github.com/ska-sa/astrokat.git to /tmp/pip-req-build-8_xnsfsr
  Running command git clone -q https://github.com/ska-sa/astrokat.git /tmp/pip-req-build-8_xnsfsr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102kB 4.4MB/s 
  Created wheel for astrokat: filename=astrokat-0.1.dev581+master.32445ac-cp37-none-any.whl size=57703 sha256=e8909978d2d67d399c40ae10ffd8c6975d3899cfb8795a1d4d8418d10ebb58c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xo0hbcgg/wheels/4c/42/c4/982808ef2b9b2d88f345d5e8b81c563a18023a782b006c3ec8
Successfully built astrokat
  Created wheel for pyephem: filename=pyephem-9.99-cp37-none-any.whl size=1570 sha256=164b10b6d91c78157ca9223e70aa37eb6c0f5fcb36a0aea8c17764117a82eab1
  Stored in directory: /root/.cache/pip/wheels/9d/c2/8d/c7117f6ac25cad94939c5dbbddbb4ba312e0b808edf11912a6
Successfull

In [2]:
!git clone https://github.com/ska-sa/astrokat.git

Cloning into 'astrokat'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 2733 (delta 105), reused 126 (delta 66), pack-reused 2548
Receiving objects: 100% (2733/2733), 5.39 MiB | 21.98 MiB/s, done.
Resolving deltas: 100% (1890/1890), done.


In [3]:
# remove between the comment after merge

In [1]:
!pip install git+https://github.com/rubyvanrooyen/astrokat.git@noncelestial_observations

  Cloning https://github.com/rubyvanrooyen/astrokat.git (to revision noncelestial_observations) to /tmp/pip-req-build-7aodmdtn
  Running command git clone -q https://github.com/rubyvanrooyen/astrokat.git /tmp/pip-req-build-7aodmdtn
  Running command git checkout -b noncelestial_observations --track origin/noncelestial_observations
  Switched to a new branch 'noncelestial_observations'
  Branch 'noncelestial_observations' set up to track remote branch 'noncelestial_observations' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102kB 4.7MB/s 
  Created wheel for astrokat: filename=astrokat-0.1.dev589+noncelestial.observations.4276b6a-cp37-none-any.whl size=61091 sha256=a41179aed5bfc6162afe894c4323f1ccb3744120a6406ecce1ba30cdeef72b49
  Stored in directory: /tmp/pip-ephem-wheel-cache-ahboyxhr/wheels/61/ae/36/9bc667824

In [2]:
!git clone --single-branch --branch noncelestial_observations https://github.com/rubyvanrooyen/astrokat.git

Cloning into 'astrokat'...
remote: Enumerating objects: 549, done.
remote: Counting objects: 100% (549/549), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 2738 (delta 374), reused 501 (delta 346), pack-reused 2189
Receiving objects: 100% (2738/2738), 5.38 MiB | 19.14 MiB/s, done.
Resolving deltas: 100% (1887/1887), done.


In [6]:
# remove between the comment after merge

In [3]:
!ln -s /content/astrokat/scripts/astrokat-catalogue2obsfile.py catalogue2obsfile.py

## User input
Upload CSV catalogue containing target coordinates for convertion to observation file

In [4]:
from google.colab import files
uploaded = files.upload()
filename = next(iter(uploaded))
print(filename)

Saving sample_targetlist_with_cals.csv to sample_targetlist_with_cals.csv
sample_targetlist_with_cals.csv


MeerKAT instrument setup for observation

In [5]:
#@markdown observation instrument product configuration
product =  'c856M4k' #@param {type:"string"}
# add lists with values allowed -- and add standard imaging modes
# text_and_dropdown = 'value' #@param ["1st option", "2nd option", "3rd option"] {allow-input: true}

#@markdown observation band
band = 'L' #@param ["L", "UHF", "X", "S"]

#@markdown averaging time per dump [sec]
integration_period = 8.0 #@param {type:"number"}

Target observation strategy.    
Tracking a target for imaging or spectral line observations.

In [6]:
#@markdown observation start LST [HH:MM] (-1 for auto)
lst = -1 #@param {type:"number"}

#@markdown default target track duration [sec]
target_duration = 300.0 #@param {type:"number"}

#@markdown maximum duration of observation [sec] (-1 for auto)
max_duration =  35400#@param {type:"number"}

Calibrator observation strategy.   
Calibrators are identified by tags in their description strings: `bpcal`, `gaincal`, `fluxcal` and `polcal` respectively

In [7]:
#@markdown primary calibrators are 'bpcal', 'fluxcal' and 'polcal'

#@markdown minimum duration to track primary calibrators [sec].  
primary_cal_duration = 180. #@param {type:"number"}

#@markdown minimum observation interval between primary calibrators [sec]  (-1 for no cadence)
primary_cal_cadence = 1800 #@param {type:"number"}


#@markdown primary calibrators are 'gaincal' and 'delaycal'

#@markdown minimum duration to track gain calibrator [sec]
secondary_cal_duration = 65. #@param {type:"number"}

Sources are specified as a catalogue of targets, with optional timing information

In [8]:
#@markdown filename for output observation file (default only displays output)
outfile = 'sample_observation_file.yaml' #@param {type:"string"}

## Convert CSV to YAML

In [9]:
import astrokat
import catalogue2obsfile as cat2obs

In [10]:
# unpack and verify user input
instrument = astrokat.obs_dict.get_instrument_dict()
instrument['product'] = None if not product else product
instrument['band'] = None if not band else band
instrument['integration_period'] = integration_period
instrument = astrokat.obs_dict.remove_none_inputs_(instrument)

def eval_none(val_):
    if val_ < 0:
        return None
    else:
        return val_

durations = astrokat.obs_dict.get_durations_dict()
max_duration = eval_none(max_duration)
durations["obs_duration"] = max_duration
durations = astrokat.obs_dict.remove_none_inputs_(durations)

lst = eval_none(lst)
primary_cal_cadence = eval_none(primary_cal_cadence)

if not outfile: outfile = None

In [11]:
# read targets from catalogue file
cat_obj = cat2obs.UnpackCatalogue(filename)
header, catalogue = cat_obj.read_catalogue(target_duration=target_duration,
                                           gaincal_duration=secondary_cal_duration,
                                           bpcal_duration=primary_cal_duration,
                                           bpcal_interval=primary_cal_cadence)

In [12]:
# create observation configuration output
obs_plan = cat2obs.BuildObservation(catalogue)
obs_plan.configure(instrument=instrument,
                   obs_duration=durations,
                   lst=lst)

In [13]:
# display observation info or write to YAML file
obs_plan.write_yaml(header=header, outfile=outfile)

In [14]:
# view yaml
!ln -s /content/astrokat/scripts/astrokat-targets.py mkatcals.py

In [15]:
import mkatcals
mkatcals.main(viewfile=outfile)

TypeError: ignored

In [16]:
mkatcals.main

<function mkatcals.main>